## HERE Traffic API Integration

### UK Regions

In [4]:
import requests
import json

# Replace with your HERE API Key
API_KEY = 'N82GXPWfciDemPYjpN8K9vb2OzPrbNYkjs48M7zbcC8'

# Function to fetch traffic data for a specific bounding box
def get_here_traffic_data(bbox):
    url = f"https://data.traffic.hereapi.com/v7/flow?locationReferencing=shape&in=bbox:{bbox}&apiKey={API_KEY}"
    response = requests.get(url)
    
    if response.status_code == 200:
        data = response.json()
        if data.get("results"):
            return data
        else:
            print("No traffic data found for the specified bounding box.")
            return None
    else:
        print(f"Error fetching data: {response.status_code}")
        return None

# Function to parse and display important traffic information
def parse_traffic_data(data):
    if data:
        for result in data["results"]:
            location = result.get("location", {})
            description = location.get("description", "No description available")
            length = location.get("length", "N/A")
            shape = location.get("shape", {})
            
            print(f"Road: {description}")
            print(f"Length: {length} meters")
            
            if "links" in shape:
                print("Coordinates of the traffic flow (lat, lng):")
                for link in shape["links"]:
                    for point in link["points"]:
                        print(f"Lat: {point['lat']}, Lng: {point['lng']}")
                print("\n")
            else:
                print("No shape data available.")
    else:
        print("No data to parse.")

# Bounding box for central London (Ladbroke Grove)
bbox = "-0.15,51.50,-0.10,51.52"

# Fetch traffic data
traffic_data = get_here_traffic_data(bbox)

# Parse and display the traffic data
parse_traffic_data(traffic_data)

Road: Covent Garden
Length: 345.0 meters
Coordinates of the traffic flow (lat, lng):
Lat: 51.51617, Lng: -0.12002
Lat: 51.51587, Lng: -0.12051
Lat: 51.51587, Lng: -0.12051
Lat: 51.51564, Lng: -0.12084
Lat: 51.51564, Lng: -0.12084
Lat: 51.51531, Lng: -0.12133
Lat: 51.51531, Lng: -0.12133
Lat: 51.51518, Lng: -0.12153
Lat: 51.51518, Lng: -0.12153
Lat: 51.51501, Lng: -0.12176
Lat: 51.51501, Lng: -0.12176
Lat: 51.51482, Lng: -0.1218
Lat: 51.51482, Lng: -0.1218
Lat: 51.51458, Lng: -0.12205
Lat: 51.51458, Lng: -0.12205
Lat: 51.51449, Lng: -0.12218
Lat: 51.51419, Lng: -0.12264
Lat: 51.51419, Lng: -0.12264
Lat: 51.51402, Lng: -0.1229
Lat: 51.51402, Lng: -0.1229
Lat: 51.51391, Lng: -0.12308
Lat: 51.51391, Lng: -0.12308
Lat: 51.51383, Lng: -0.12319


Road: Westminster
Length: 1524.0 meters
Coordinates of the traffic flow (lat, lng):
Lat: 51.49505, Lng: -0.1011
Lat: 51.49513, Lng: -0.10132
Lat: 51.49516, Lng: -0.10142
Lat: 51.4952, Lng: -0.10156
Lat: 51.4952, Lng: -0.10156
Lat: 51.49528, Lng: -0.1

In [30]:
import os
import requests
import json
import time

# Replace with your HERE API Key
API_KEY = 'N82GXPWfciDemPYjpN8K9vb2OzPrbNYkjs48M7zbcC8'

# Base directory where you want to save your raw data
BASE_DIR = '../data/raw'

# Ensure directory exists
def ensure_dir(directory):
    if not os.path.exists(directory):
        os.makedirs(directory)

# Function to fetch traffic data (flow or incidents)
def get_here_data(bbox, data_type="flow"):
    url = f"https://data.traffic.hereapi.com/v7/{data_type}?locationReferencing=shape&in=bbox:{bbox}&apiKey={API_KEY}"
    response = requests.get(url)
    
    if response.status_code == 200:
        data = response.json()
        if data.get("results"):
            return data
        else:
            print(f"No {data_type} data found for the specified bounding box.")
            return None
    else:
        print(f"Error fetching {data_type} data: {response.status_code}")
        return None

# Function to save data into a JSON file inside the correct folder
def save_data_to_file(data, data_type):
    # Define folder based on data type (flow, incidents, advanced traffic)
    folder = os.path.join(BASE_DIR, data_type)
    
    # Ensure the folder exists
    ensure_dir(folder)
    
    # Generate the file name based on the current timestamp
    filename = f"{data_type}_data_{time.strftime('%Y%m%d_%H%M%S')}.json"
    
    # Full path to save the file
    filepath = os.path.join(folder, filename)
    
    # Save the data into the file
    with open(filepath, 'w') as outfile:
        json.dump(data, outfile, indent=4)
    
    print(f"Data saved to {filepath}")

# Function to automate fetching and saving updates
def fetch_and_save_updates(bbox, interval_flow=60, interval_incidents=120):
    while True:
        # Fetch flow data
        flow_data = get_here_data(bbox, data_type="flow")
        if flow_data:
            save_data_to_file(flow_data, "flow")

        # Wait for the interval before fetching incident data
        time.sleep(interval_flow)

        # Fetch incidents data
        incident_data = get_here_data(bbox, data_type="incidents")
        if incident_data:
            save_data_to_file(incident_data, "incidents")

        # Wait before the next fetch cycle
        time.sleep(interval_incidents)

# Bounding box for central London
bbox = "-0.15,51.50,-0.10,51.52"

# Fetch and save real-time traffic data
fetch_and_save_updates(bbox, interval_flow=60, interval_incidents=120)


Data saved to ../data/raw/flow/flow_data_20241024_222157.json


KeyboardInterrupt: 

In [17]:
import os
import requests
import json
import time

# Replace with your HERE API Key
API_KEY = 'N82GXPWfciDemPYjpN8K9vb2OzPrbNYkjs48M7zbcC8'
BASE_DIR = '../data/raw'

# Ensure directory exists
def ensure_dir(directory):
    if not os.path.exists(directory):
        os.makedirs(directory)

# Save data to the appropriate folder
def save_data_to_file(data, folder):
    # Ensure the folder exists
    folder_path = os.path.join(BASE_DIR, folder)
    ensure_dir(folder_path)
    
    # Generate file name based on timestamp
    filename = f"data_{time.strftime('%Y%m%d_%H%M%S')}.json"
    filepath = os.path.join(folder_path, filename)
    
    # Save to file
    with open(filepath, 'w') as outfile:
        json.dump(data, outfile, indent=4)
    print(f"Data saved to {filepath}")

# Fetch general flow or incident data
def get_here_data(bbox, data_type="flow"):
    url = f"https://data.traffic.hereapi.com/v7/{data_type}?locationReferencing=shape&in=bbox:{bbox}&apiKey={API_KEY}"
    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()
        if data.get("results"):
            return data
        else:
            print(f"No {data_type} data found for the specified bounding box.")
            return None
    else:
        print(f"Error fetching {data_type} data: {response.status_code}")
        return None

# Fetch lane-level flow data
def get_here_lane_level_flow(bbox):
    url = f"https://data.traffic.hereapi.com/v7/flow?locationReferencing=shape&in=bbox:{bbox}&advancedFeatures=lanes&apiKey={API_KEY}"
    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()
        if data.get("results"):
            return data
        else:
            print(f"No lane-level flow data found for the specified bounding box.")
            return None
    else:
        print(f"Error fetching lane-level flow data: {response.status_code}")
        return None

# Fetch deep coverage traffic data
def get_here_deep_coverage_data(bbox):
    url = f"https://data.traffic.hereapi.com/v7/flow?locationReferencing=shape&in=bbox:{bbox}&advancedFeatures=deepCoverage&apiKey={API_KEY}"
    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()
        if data.get("results"):
            return data
        else:
            print(f"No deep coverage data found for the specified bounding box.")
            return None
    else:
        print(f"Error fetching deep coverage data: {response.status_code}")
        return None

# Filter flow data based on jamFactor and traversability
def filter_flow_data(data):
    filtered_data = []
    for result in data.get("results", []):
        current_flow = result.get("currentFlow", {})
        jam_factor = current_flow.get("jamFactor", 0)
        traversability = current_flow.get("traversability", "open")
        
        # Filter out roads with high congestion (e.g., jamFactor > 7) or road closures
        if jam_factor > 7 or traversability == "closed":
            filtered_data.append(result)
    
    return filtered_data

# Fetch and save flow, incident, and advanced data types
def fetch_and_save_all_data(bbox):
    # Fetch Flow data
    flow_data = get_here_data(bbox, data_type="flow")
    if flow_data:
        # Save raw flow data
        save_data_to_file(flow_data, "flow/raw")
        
        # Filter data based on jamFactor and traversability
        filtered_flow_data = filter_flow_data(flow_data)
        if filtered_flow_data:
            save_data_to_file(filtered_flow_data, "flow/high_congestion_or_closed")

    # Fetch Incident data
    incident_data = get_here_data(bbox, data_type="incidents")
    if incident_data:
        save_data_to_file(incident_data, "incidents/raw")

    # Fetch Lane-Level flow data
    lane_flow_data = get_here_lane_level_flow(bbox)
    if lane_flow_data:
        save_data_to_file(lane_flow_data, "flow/lane_level")

    # Fetch Deep Coverage flow data
    deep_coverage_data = get_here_deep_coverage_data(bbox)
    if deep_coverage_data:
        save_data_to_file(deep_coverage_data, "flow/deep_coverage")

# Bounding box for central London
bbox = "-0.15,51.50,-0.10,51.52"

# Fetch and save all data
fetch_and_save_all_data(bbox)

Data saved to ../data/raw/flow/raw/data_20241024_220604.json
Data saved to ../data/raw/flow/high_congestion_or_closed/data_20241024_220604.json
Data saved to ../data/raw/incidents/raw/data_20241024_220605.json
Data saved to ../data/raw/flow/lane_level/data_20241024_220605.json
Data saved to ../data/raw/flow/deep_coverage/data_20241024_220605.json


/opt/anaconda3/envs/bigdata_ml/lib/python3.8/json/decoder.py:353: RuntimeWarning: coroutine 'fetch_and_save_all_data_async' was never awaited
  obj, end = self.scan_once(s, idx)


In [29]:
import os
import requests
import json
import time
import pandas as pd

# Replace with your HERE API Key
API_KEY = 'N82GXPWfciDemPYjpN8K9vb2OzPrbNYkjs48M7zbcC8'
BASE_DIR = '../data/raw'

# Ensure directory exists
def ensure_dir(directory):
    if not os.path.exists(directory):
        os.makedirs(directory)

# Save data to JSON and CSV formats
def save_data_to_file(data, folder, file_prefix):
    # Ensure the folder exists
    folder_path = os.path.join(BASE_DIR, folder)
    ensure_dir(folder_path)
    
    # Generate file name based on timestamp
    timestamp = time.strftime('%Y%m%d_%H%M%S')
    
    # Save to JSON file
    json_filename = f"{file_prefix}_data_{timestamp}.json"
    json_filepath = os.path.join(folder_path, json_filename)
    with open(json_filepath, 'w') as outfile:
        json.dump(data, outfile, indent=4)
    print(f"Data saved to {json_filepath}")
    
    # Flatten and save to CSV if there is 'results' key
    if "results" in data:
        df = pd.json_normalize(data["results"])
        csv_filename = f"{file_prefix}_data_{timestamp}.csv"
        csv_filepath = os.path.join(folder_path, csv_filename)
        df.to_csv(csv_filepath, index=False)
        print(f"Flattened data saved to {csv_filepath}")

# Fetch general flow or incident data
def get_here_data(bbox, data_type="flow"):
    url = f"https://data.traffic.hereapi.com/v7/{data_type}?locationReferencing=shape&in=bbox:{bbox}&apiKey={API_KEY}"
    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()
        if data.get("results"):
            return data
        else:
            print(f"No {data_type} data found for the specified bounding box.")
            return None
    else:
        print(f"Error fetching {data_type} data: {response.status_code}")
        return None

# Fetch lane-level flow data
def get_here_lane_level_flow(bbox):
    url = f"https://data.traffic.hereapi.com/v7/flow?locationReferencing=shape&in=bbox:{bbox}&advancedFeatures=lanes&apiKey={API_KEY}"
    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()
        if data.get("results"):
            return data
        else:
            print(f"No lane-level flow data found for the specified bounding box.")
            return None
    else:
        print(f"Error fetching lane-level flow data: {response.status_code}")
        return None

# Fetch deep coverage traffic data
def get_here_deep_coverage_data(bbox):
    url = f"https://data.traffic.hereapi.com/v7/flow?locationReferencing=shape&in=bbox:{bbox}&advancedFeatures=deepCoverage&apiKey={API_KEY}"
    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()
        if data.get("results"):
            return data
        else:
            print(f"No deep coverage data found for the specified bounding box.")
            return None
    else:
        print(f"Error fetching deep coverage data: {response.status_code}")
        return None

# Filter flow data based on jamFactor and traversability
def filter_flow_data(data):
    filtered_data = []
    for result in data.get("results", []):
        current_flow = result.get("currentFlow", {})
        jam_factor = current_flow.get("jamFactor", 0)
        traversability = current_flow.get("traversability", "open")
        
        # Filter out roads with high congestion (e.g., jamFactor > 7) or road closures
        if jam_factor > 7 or traversability == "closed":
            filtered_data.append(result)
    
    return filtered_data

# Fetch and save flow, incident, and advanced data types
def fetch_and_save_all_data(bbox):
    # Fetch Flow data
    flow_data = get_here_data(bbox, data_type="flow")
    if flow_data:
        # Save raw flow data
        save_data_to_file(flow_data, "flow/raw", "flow")
        
        # Filter data based on jamFactor and traversability
        filtered_flow_data = filter_flow_data(flow_data)
        if filtered_flow_data:
            save_data_to_file({"results": filtered_flow_data}, "flow/high_congestion_or_closed", "filtered_flow")

    # Fetch Incident data
    incident_data = get_here_data(bbox, data_type="incidents")
    if incident_data:
        save_data_to_file(incident_data, "incidents/raw", "incident")

    # Fetch Lane-Level flow data
    lane_flow_data = get_here_lane_level_flow(bbox)
    if lane_flow_data:
        save_data_to_file(lane_flow_data, "flow/lane_level", "lane_flow")

    # Fetch Deep Coverage flow data
    deep_coverage_data = get_here_deep_coverage_data(bbox)
    if deep_coverage_data:
        save_data_to_file(deep_coverage_data, "flow/deep_coverage", "deep_coverage")

# Bounding box for central London
bbox = "-0.15,51.50,-0.10,51.52"

# Fetch and save all data
fetch_and_save_all_data(bbox)

Data saved to ../data/raw/flow/raw/flow_data_20241024_222043.json
Flattened data saved to ../data/raw/flow/raw/flow_data_20241024_222043.csv
Data saved to ../data/raw/flow/high_congestion_or_closed/filtered_flow_data_20241024_222043.json
Flattened data saved to ../data/raw/flow/high_congestion_or_closed/filtered_flow_data_20241024_222043.csv
Data saved to ../data/raw/incidents/raw/incident_data_20241024_222043.json
Flattened data saved to ../data/raw/incidents/raw/incident_data_20241024_222043.csv
Data saved to ../data/raw/flow/lane_level/lane_flow_data_20241024_222043.json
Flattened data saved to ../data/raw/flow/lane_level/lane_flow_data_20241024_222043.csv
Data saved to ../data/raw/flow/deep_coverage/deep_coverage_data_20241024_222044.json
Flattened data saved to ../data/raw/flow/deep_coverage/deep_coverage_data_20241024_222044.csv


In [31]:
flow_df = pd.read_csv('/Users/emillianlandu/Desktop/traffic_intelligent_platform/main/data/raw/flow/deep_coverage/deep_coverage_data_20241024_222044.csv')
accident_df = pd.read_csv('/Users/emillianlandu/Desktop/traffic_intelligent_platform/main/data/raw/incident_data_20241024_184850.json')

In [32]:
flow_df.head()

,location.length,location.shape.links,currentFlow.freeFlow,currentFlow.jamFactor,currentFlow.traversability,currentFlow.junctionTraversability,location.description,currentFlow.speed,currentFlow.speedUncapped,currentFlow.confidence,currentFlow.subSegments,currentFlow.jamTendency
0,61.0,"[{'points': [{'lat': 51.51306, 'lng': -0.10695...",0.000000,10.0,closed,intermediateClosedEdgeOpen,NaN,NaN,NaN,NaN,NaN,NaN
1,16.0,"[{'points': [{'lat': 51.51389, 'lng': -0.10736...",0.000000,10.0,closed,intermediateClosedEdgeOpen,NaN,NaN,NaN,NaN,NaN,NaN
2,814.0,"[{'points': [{'lat': 51.5186, 'lng': -0.10532}...",5.555556,0.4,open,NaN,A3211 at City,4.444445,4.444445,0.92,NaN,NaN
3,5189.0,"[{'points': [{'lat': 51.51614, 'lng': -0.2158}...",9.166667,0.0,open,NaN,Regents Park,10.277778,10.277778,0.99,"[{'length': 3569.0, 'speed': 11.666667, 'speed...",NaN
4,29.0,"[{'points': [{'lat': 51.51317, 'lng': -0.14067...",4.722222,2.1,open,NaN,Soho,3.611111,3.611111,0.70,NaN,NaN
